##### Imports

In [44]:
import os
import time
import ee
import requests
import shutil, sys
import logging
import multiprocessing
from retry import retry

### Earth Engine Authentication

In [45]:
#ee.Authenticate()

In [46]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

### Data Loading

In [47]:
def mask_L8_SR (img):
    '''takes in an image and sets a mask for its low quality pixels'''
    qa = img.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3)
    shadow = qa.bitwiseAnd(1 << 4)
    Mask = (cloud.neq(0)).Or(shadow.neq(0)).uint8()
    return  img.updateMask(Mask.eq(0))


#### Year selection

In [48]:
# ---> ENTER Year BELOW <---
YEAR = 2008
print(f'YEAR: {YEAR}')

YEAR: 2008


#### Country selection

In [49]:
# ---> ENTER COUNTRY BELOW <---
COUNTRY = "Nigeria"

countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
border = countries.filter(ee.Filter.eq('ADM0_NAME', COUNTRY))
img = ee.Image(1).clip(border)

pointsROI = img.stratifiedSample(numPoints = 1000000,
                                classBand = 'constant',
                                region = border,
                                scale = 10000,
                                geometries = True)

print(f'COUNTRY: {COUNTRY}, POINTS: {pointsROI.size().getInfo()}')

COUNTRY: Nigeria, POINTS: 9294


#### Datasets

In [50]:
# Landsat-8 (input)
L5 = (ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
                .filterDate(f'{YEAR}-01-01',f'{YEAR + 1}-01-01')
                .select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7']
                        ,['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])).median() 
# .map(lambda x: mask_L8_SR(x)

In [51]:
# Nightlight (input)
DMSP = (ee.ImageCollection("BNU/FGS/CCNL/v1")
                  .filterDate(f'{YEAR}-01-01',f'{YEAR + 1}-01-01')
                  .select(['b1'],['NL'])).median()


#### Data retrieval

In [52]:
# specify output directory
OUT_DIR = f'/mimer/NOBACKUP/groups/globalpoverty1/albin_and_albin/lagos/input'

# create directory if it does not exist
if not os.path.isdir(OUT_DIR):
        os.makedirs(OUT_DIR, 0o775)
        print(f'Directory created (mimer): ..{OUT_DIR.partition("globalpoverty1")[-1]}')

In [53]:
@retry(tries=10, delay=1, backoff=2)
def get_image_tile(point):
    '''Retrieve image tile at given point, scale and dimension.  Write to directory'''
    
    # get ID and point-coordinates
    ID = point['id']
    point = ee.Geometry.Point(point['geometry']['coordinates'])
    
    # set up rectangular bound around point
    ROI = point.buffer(500*10).bounds()  # tile dim: 1000*1000px (1px=10m)
    
    # images to retrieve
    imgLandsat = L5.clip(ROI)
    imgNL = DMSP.clip(ROI)
    
    # concatenate input
    #imageInput = ee.Image.cat([imgLandsat, imgNL, imgSMOD, imgTarget_esa, imgTarget_esri])
    imageInput = ee.Image.cat([imgLandsat, imgNL])

    # fetch the URL from which to download the image.
    url = imageInput.float().getDownloadUrl({
        'scale': 10,
        'dimensions': '1000x1000',
        'format': 'GEO_TIFF'
    })
    r = requests.get(url) # send get request
    
    # save retrieved tile
    if r.status_code == 200:  # HTTP GET: 200 OK
        filename = OUT_DIR + f'/tile_{ID}.tif_{YEAR}_test'
        with open(filename, 'wb') as out_file:
              out_file.write(r.content)
    # retry, get request failed
    else:
        #print(f'{r.status_code}: {r.reason}')
        raise HTTPException(status_code=r.status_code, detail=r.reason)
    
    return r.content


In [ ]:
# make a list of the points
image_points = pointsROI.toList(pointsROI.size().getInfo()).getInfo()

# point indices (found by visual inspection in ee)
im_indices = [483, 5412, 3324, 
              7601, 4497, 4254, 
              3545, 7904, 2557]

for im_idx in im_indices:    
    
    # retrieve one point at a time
    point_to_download = image_points[im_idx]

    # get corresponding image tiles
    get_image_tile(point_to_download)

    print(f'redownload complete: {COUNTRY}, {YEAR}, tile_{point_to_download["id"]}')

redownload complete: Nigeria, 2008, tile_483
redownload complete: Nigeria, 2008, tile_5412
redownload complete: Nigeria, 2008, tile_3324


In [ ]:
print(point_to_download["id"])